### **特定のモデルの予測値を書き出す**

In [2]:
# データ加工・処理・分析モジュール
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

import os
import random

%matplotlib inline

import time

In [3]:
def set_time(dataframe, col_name):
    '''
    to_datetimeを使うための前処理
    '''
    dataframe[col_name] = dataframe[col_name].map(lambda x : transform_time(x))
    return dataframe

In [4]:
def add_time_sincos(input_data, key="datetime"):
    '''
    時間や月のデータを加える
    '''
    input_data['hour'] = input_data['datetime'].map(lambda x: int(x.hour))
    input_data['minute'] = input_data['datetime'].map(lambda x: int(x.minute))
    input_data['month'] = input_data['datetime'].map(lambda x: int(x.month))
    input_data['month_cos'] = input_data['month'].map(lambda x: np.cos(np.pi * x / 12))
    input_data['month_sin'] = input_data['month'].map(lambda x: np.sin(np.pi * x / 12))
    input_data['hour_cos'] = input_data['hour'].map(lambda x: np.cos(np.pi * x / 24))
    input_data['hour_sin'] = input_data['hour'].map(lambda x: np.sin(np.pi * x / 24))
    input_data['year'] = input_data['datetime'].map(lambda x: int(x.year))
    input_data = input_data.drop(['hour', 'month', 'minute'], axis=1)
    return input_data

In [5]:
def transform_time(x):
    '''
    set_time内で使う関数
    to_datetimeで24時をサポートしないので00に変更する処理
    '''
    str_x = str(x)
    res = ''
    if str(x)[8:10] == '24':
        res = str_x[0:4] + '-' + str_x[4:6] + '-' + str_x[6:8] + ' 00:'+str_x[10:12] 
    else:
        res = str_x[0:4] + '-' + str_x[4:6] + '-' + str_x[6:8] + ' '+ str_x[8:10] +':'+str_x[10:12]
    return res

In [6]:
def drop_nan(X, Y):
    '''
    正解データがnanであるデータの組を削除
    '''
    mask = np.isnan(Y)
    X = X[~mask]
    Y = Y[~mask]
    return X, Y

In [7]:
def to_zero(prediction):
    '''
    2012年から2015年の間で一度も発電量がなかった時間帯を0と予測するようにする
    '''
    mask = np.array([0,0,0,0,0,0,0,0,0,0,1,1,
                    1,1,1,1,1,1,1,1,1,1,1,1,
                    1,1,1,1,1,1,1,1,1,1,1,1,
                    1,1,0,0,0,0,0,0,0,0,0,0])
    all_mask = np.array([0,0,0,0,0,0,0,0,0,0,1,1,
                        1,1,1,1,1,1,1,1,1,1,1,1,
                        1,1,1,1,1,1,1,1,1,1,1,1,
                        1,1,0,0,0,0,0,0,0,0,0,0])
    length = int(len(prediction)/48)
    
    for i in range(length-1):
        all_mask = np.r_[all_mask, mask]
    
    return prediction * all_mask

In [8]:
# 発電量データ
all_output_30 = pd.read_csv('data/processed_data/out_put.tsv', delimiter = '\t')
all_output_30['datetime'] = all_output_30['datetime'].map(lambda x : pd.to_datetime(x))

In [1]:
target_place = 1

In [ ]:
# いろんなモデルの予測値の集合(ほんちゃん)
test_prediction = pd.read_csv("data/predicted_data/predict_SOLA0"+str(target_place)+".tsv", delimiter="\t")
test_prediction['datetime'] = test_prediction['datetime'].map(lambda x : pd.to_datetime(x))

In [ ]:
test_prediction.columns

In [ ]:
keys = ["targetplace_1_limited_scaled_kwh_all_wfc_sigmoid_hidden_100_1000_1-3folds",
        "targetplace_1_limited_scaled_kwh_all_wfc_sigmoid_hidden_100_1000_2-3folds",
        "targetplace_1_limited_scaled_kwh_all_wfc_sigmoid_hidden_100_1000_3-3folds"]
tmp = test_prediction[keys]
tmp = tmp.mean(axis=1)

In [ ]:
# 2016/01/01 00:00 ~ 2017/3/31 23:30の予測データを書き出す
predict_data["blending_"+str(target_place)] = tmp

In [1]:
target_place = 2

In [ ]:
# いろんなモデルの予測値の集合(ほんちゃん)
test_prediction = pd.read_csv("data/predicted_data/predict_SOLA0"+str(target_place)+".tsv", delimiter="\t")
test_prediction['datetime'] = test_prediction['datetime'].map(lambda x : pd.to_datetime(x))

In [ ]:
test_prediction.columns

In [ ]:
keys = ["targetplace_1_limited_scaled_kwh_all_wfc_sigmoid_hidden_100_1000_1-3folds",
        "targetplace_1_limited_scaled_kwh_all_wfc_sigmoid_hidden_100_1000_2-3folds",
        "targetplace_1_limited_scaled_kwh_all_wfc_sigmoid_hidden_100_1000_3-3folds"]
tmp = test_prediction[keys]
tmp = tmp.mean(axis=1)

In [ ]:
# 2016/01/01 00:00 ~ 2017/3/31 23:30の予測データを書き出す
predict_data["blending_"+str(target_place)] = tmp

In [1]:
target_place = 3

In [ ]:
# いろんなモデルの予測値の集合(ほんちゃん)
test_prediction = pd.read_csv("data/predicted_data/predict_SOLA0"+str(target_place)+".tsv", delimiter="\t")
test_prediction['datetime'] = test_prediction['datetime'].map(lambda x : pd.to_datetime(x))

In [ ]:
test_prediction.columns

In [ ]:
keys = ["targetplace_1_limited_scaled_kwh_all_wfc_sigmoid_hidden_100_1000_1-3folds",
        "targetplace_1_limited_scaled_kwh_all_wfc_sigmoid_hidden_100_1000_2-3folds",
        "targetplace_1_limited_scaled_kwh_all_wfc_sigmoid_hidden_100_1000_3-3folds"]
tmp = test_prediction[keys]
tmp = tmp.mean(axis=1)

In [ ]:
# 2016/01/01 00:00 ~ 2017/3/31 23:30の予測データを書き出す
predict_data["blending_"+str(target_place)] = tmp

In [ ]:
def minus_to_zero(x):
    if x < 0:
        return 0
    else:
        return x

In [ ]:
predict_data["blending_1"] = predict_data["blending_1"].map(lambda x: minus_to_zero(x))
predict_data["blending_2"] = predict_data["blending_2"].map(lambda x: minus_to_zero(x))
predict_data["blending_3"] = predict_data["blending_3"].map(lambda x: minus_to_zero(x))

In [ ]:
predict_data

In [ ]:
predict_data.to_csv('data/predicted_data/simple_blending_prediction.tsv', sep = '\t', index=False)